In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import joblib

In [2]:
news_df = pd.read_csv('/content/news.csv')
rec_items_df = pd.read_csv('/content/rec_items.csv')
rec_feedback_df = pd.read_csv('/content/rec_feedback.csv')
users_df = pd.read_csv('/content/users.csv')
rec_users_df = pd.read_csv('/content/rec_users.csv')

In [3]:
# Displays the first 5 rows of the news dataset
news_df.head()

,id,title,description,published_date,breaking_news,blob_image,source_url,created_at,updated_at,published_at,created_by_id,updated_by_id,shares,comment_count,type
0,23996,අද ඩොලරයේ අගය,ශ්‍රී ලංකා මහ බැංකුව විසින් අද (13) දින නිකුත්...,2025-02-13 12:10:32.919,False,NaN,https://www.hirunews.lk/396721/%E0%B6%85%E0%B6...,2025-02-13 09:43:07.843,2025-02-13 09:43:12.403,2025-02-13 09:43:12.131,7.0,7.0,0,0,News
1,23995,‘ක්ලීන් ශ‍්‍රී ලංකා ලිඛිතව තියෙනවා.’ මාලිමා මන...,‘ක්‍ලීන් ශ්‍රී ලංකා’ වැඩපිළිවෙල යනු කුමක්දැයි ...,2025-02-13 09:30:00,False,NaN,https://lankacnews.com/%e0%b6%9a%e0%b7%8a%e0%b...,2025-02-13 09:42:37.114,2025-02-13 09:42:47.003,NaN,NaN,NaN,0,0,News
2,23994,පාපන්දු ගෝල කණුවක් කඩා වැටී පාසැල් සිසුවෙකු ජී...,පාසැල් ක්‍රීඩාගංනයක තිබූ පාපන්දු ගෝල කණුවක් කඩ...,2025-02-13 11:10:47.133,False,NaN,https://www.hirunews.lk/396719/%E0%B6%B4%E0%B7...,2025-02-13 09:33:20.185,2025-02-13 09:33:23.796,2025-02-13 09:33:23.747,7.0,7.0,0,0,News
3,23993,සුජීව සේනසිංහගේ මූලික අයිතිවාසිකම් පෙත්සම විභා...,තමන්ට එරෙහිව අපරාධ පරීක්ෂණ දෙපාර්තමේන්තුව විසි...,2025-02-13 10:10:29.598,False,NaN,https://www.hirunews.lk/396716/%E0%B7%83%E0%B7...,2025-02-13 09:31:55.442,2025-02-13 09:31:59.501,2025-02-13 09:31:59.41,7.0,7.0,0,0,News
4,23992,නීති විරෝධී ධීවර දැල් දෙසීය පනහක් නීතියේ රැහැනට,"ශ්‍රී ලංකා නාවික හමුදාව, කිලිනොච්චිය මුද්දලම්ප...",2025-02-13 10:58:56.639,False,NaN,https://www.dinamina.lk/2025/02/13/lawnorder/1...,2025-02-13 09:29:23.368,2025-02-13 09:29:26.299,2025-02-13 09:29:26.258,7.0,7.0,0,0,News


In [4]:
# Displays the data types of each column
news_df.dtypes

,0
id,int64
title,object
description,object
published_date,object
breaking_news,bool
blob_image,float64
source_url,object
created_at,object
updated_at,object
published_at,object


In [5]:
news_df.columns

Index(['id', 'title', 'description', 'published_date', 'breaking_news',
       'blob_image', 'source_url', 'created_at', 'updated_at', 'published_at',
       'created_by_id', 'updated_by_id', 'shares', 'comment_count', 'type'],
      dtype='object')

In [6]:
# Prints the number of missing (null) values in each column
print(news_df.isnull().sum())

id                   0
title                0
description          0
published_date       0
breaking_news        0
blob_image        1000
source_url           0
created_at           0
updated_at           0
published_at        35
created_by_id      382
updated_by_id       35
shares               0
comment_count        0
type                 0
dtype: int64


In [7]:
# Removes the specified columns from the DataFrame
news_df.drop(columns=["blob_image", "updated_by_id", "created_by_id", "shares", "comment_count", "type"], inplace=True)

In [8]:
# Fills missing (NaN) values in a column with the most frequently occurring value (mode).

news_df['published_at'] = news_df['published_at'].fillna(news_df['published_at'].mode()[0])

In [9]:
print(news_df.isnull().sum())

id                0
title             0
description       0
published_date    0
breaking_news     0
source_url        0
created_at        0
updated_at        0
published_at      0
dtype: int64


In [10]:
# Preprocess data (combine title and description for better recommendation)
news_df['combined'] = news_df['title'] + ' ' + news_df['description']

In [11]:
# Initialize TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

In [12]:
sinhala_stop_words = [
    "අතර", "ඉන්", "එක", "එය", "ඔබ", "ඔයා", "ඔහු", "ඔවුන්", "ඕනෑ", "ආදිය", "ඇයි", "ඇතුලත", "ඉන්පසු",
    "ඉස්සර", "එක්", "ඔය", "ඔයාලා", "ඔයාව", "උදාහරණ", "උපුටා", "ඉදිරියට", "එදා", "ඒ", "ඒවා", "ඕන", "ඔක්කෝම",
    "ඔවුන්", "අප", "අපේ", "ඇතුළත්", "ඇත", "ඊයේ", "ඔබට", "ඔබගේ", "එහි", "එන්න", "ඕක", "ඉහළ", "ඔබේ", "අය", "ඔව්",
    "අද", "ඉතා", "ඉතාම", "ඉහත", "අවශ්‍ය", "ඔයාලට"
]
tfidf = TfidfVectorizer(stop_words=sinhala_stop_words)

In [13]:
# Fit the model and transform the 'combined' column (title + description)
tfidf_matrix = tfidf.fit_transform(news_df['combined'])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['අවශ', 'ආද', 'ඇය', 'ඉත', 'ඉද', 'ඉන', 'ඉස', 'ඊය', 'උද', 'උප', 'එද', 'එන', 'එහ', 'ඒව', 'ඔක', 'ඔබග', 'ඔව', 'ඔහ', 'පස', 'යට', 'ලට', 'ලත', 'සර', 'හරණ', 'ළත'] not in stop_words.
  warnings.warn(


In [14]:
# Compute cosine similarity between articles
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [15]:
from sklearn.neighbors import NearestNeighbors

# Initialize KNN model for approximate nearest neighbors
knn = NearestNeighbors(metric="cosine", algorithm="brute")
knn.fit(tfidf_matrix)  # Fit the model with the TF-IDF feature vectors

NearestNeighbors(algorithm='brute', metric='cosine')

In [16]:
# Find similar articles using KNN
def get_user_recommendations(user_id, top_n=5, similarity_threshold=0.1):
    # Get articles the user has interacted with
    user_interactions = rec_feedback_df[rec_feedback_df['user_id'] == user_id]

    if user_interactions.empty:
        return "No interactions found for this user."

    # Get article indices
    interacted_article_ids = user_interactions['item_id'].unique()
    article_indices = news_df[news_df['id'].isin(interacted_article_ids)].index.tolist()

    if len(article_indices) == 0:
        return "No articles found for user interactions."

    # Get similar articles using KNN
    distances, indices = knn.kneighbors(tfidf_matrix[article_indices], n_neighbors=top_n+5)

    # Flatten results and sort by similarity (lower distance = higher similarity)
    sim_scores = sorted(zip(indices.flatten(), distances.flatten()), key=lambda x: x[1])

    # Filter and return recommendations
    recommended_articles = []
    for idx, dist in sim_scores:
        if news_df.iloc[idx]['id'] not in interacted_article_ids and dist <= (1 - similarity_threshold):
            recommended_articles.append(news_df.iloc[idx][['id', 'title', 'description']])
        if len(recommended_articles) == top_n:
            break

    return pd.DataFrame(recommended_articles)

In [17]:
# Save the TF-IDF vectorizer and cosine similarity matrix
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')  # Save the TF-IDF vectorizer
joblib.dump(cosine_sim, 'cosine_similarity_matrix.pkl')  # Save the cosine similarity matrix

['cosine_similarity_matrix.pkl']

In [18]:
# Load the saved TF-IDF vectorizer and cosine similarity matrix
tfidf = joblib.load('tfidf_vectorizer.pkl')
cosine_sim = joblib.load('cosine_similarity_matrix.pkl')

In [19]:
# Test the recommendation system with user_id = 417
user_id = 417
recommended_articles = get_user_recommendations(user_id, top_n=5)

# Display the recommended articles
print("Recommended articles for user_id:", user_id)
print(recommended_articles)

Recommended articles for user_id: 417
        id                                              title  \
107  23888                           හෙට විදුලි කප්පාදුවක් නෑ   
75   23921  ලස්සන ලෝකයක් සඳහා ජනපතිගෙන් ලෝක නායකයින්ට ආරාධ...   
30   23966          නාමල්ට එරෙහි නඩුවකට නීතිපති උපදෙස් විමසයි   
158  23835  ට්‍රම්ප් අමෙරිකාවට ගෙන්වන ඇලුමිනියම් සහ වානේ ම...   
70   23926  ආදරවන්තයින්ගේ දිනය සමරන්න කළින් පොලිසියෙන් විශ...   

                                           description  
107  හෙට දිනයේ විදුලිය විසන්ධි කිරීමක් සිදු නොකරන බ...  
75   දේශසීමා ඉක්මවා යන අභියෝගයන්ට මුහුණදීමට ගෝලීය ප...  
30   පාර්ලිමේන්තු මන්ත්‍රී නාමල් රාජපක්ෂ මහතා ඇතුළු...  
158  අමෙරිකාවට ආනයනය කරන සියලු‍ම වානේ සහ ඇලු‍මිනියම...  
70   ආදරවන්තයින්ගේ දිනය හෙටට (14)ට යෙදී තිබෙනවා. ඒ ...  


In [20]:
user_id = 2329
recommended_articles = get_user_recommendations(user_id, top_n=5)

# Display the recommended articles
print(recommended_articles)

        id                                              title  \
719  23269  ලාංකිකයින් 3065ක් අමෙරිකාවෙන් පිටුවහල් කිරීමට ...   
504  23484            ඉන්ධන සැපයූ සමාගමක් ලංකාවෙන් ඉවත්ව යයි.   
145  23848                   ට්‍රම්ප්ගෙන් 25%ක අලුත් බද්දක්\n   
469  23521           සංචාරකයින්ගේ පැමිණීමේ වාර්තාගත වැඩිවීමක්   
167  23826        ජනපතිගේ ඩුබායි නිල සංචාරයේ දෙවැනි දිනය අදයි   

                                           description  
719  නව අමෙරිකානු පරිපාලනය විසින් පිටුවහල් කිරීමට න...  
504  මෙරට ඉන්ධන වෙලදාම සදහා පැමිණි ඕස්ට්‍රේලියාවේ ...  
145  අමෙරිකාවට ආනයනය කරන සියලු‍ම වානේ සහ ඇලු‍මිනියම...  
469  මෙරටට පැමිණි සංචාරකයින්ගේ වාර්තාගත වැඩි වීමක් ...  
167   ජනාධිපති අනුර කුමාර දිසානායක මහතාගේ එක්සත් අර...  


In [21]:
user_id = 1376
recommended_articles = get_user_recommendations(user_id, top_n=5)

# Display the recommended articles
print(recommended_articles)

        id                                              title  \
241  23751              හිටපු අගමැති ඛාන් වෙනුවෙන් විරෝධතාවක්   
230  23762  යළි විදුලි කප්පාදු කාල සටහනක් එයි. - නොරොච්චලේ...   
167  23826        ජනපතිගේ ඩුබායි නිල සංචාරයේ දෙවැනි දිනය අදයි   
265  23727               ජනපති අද එක්සත් අරාබි එමිර් රාජ්‍යයට   
254  23738               ජනපති අද එක්සත් අරාබි එමීර් රාජ්‍යයට   

                                           description  
241  විවිධ චෝදනා රැසකට වරදකරු කර ඇති පාකිස්තානයේ හි...  
230  ඉදිරි දින කිහිපයක් සඳහා විදුලිය කප්පාදු කිරීමට...  
167   ජනාධිපති අනුර කුමාර දිසානායක මහතාගේ එක්සත් අර...  
265  තෙදින නිල සංචාරයක් සඳහා ජනාධිපති අනුර කුමාර දි...  
254  ජනාධිපති අනුර කුමාර දිසානායක මහතා අද (10) එක්ස...  


In [22]:
user_id = 1045
recommended_articles = get_user_recommendations(user_id, top_n=5)
print("\nRecommended articles for user_id:", user_id)
print(recommended_articles)


Recommended articles for user_id: 1045
        id                                              title  \
653  23335           රාජ්‍ය සේවක වැටුපට අයවැයෙන් වෙන්කරන මුදල   
230  23762  යළි විදුලි කප්පාදු කාල සටහනක් එයි. - නොරොච්චලේ...   
70   23926  ආදරවන්තයින්ගේ දිනය සමරන්න කළින් පොලිසියෙන් විශ...   
829  23156  ආණ්ඩුව වෙනස් වුනාට හිතූපැතූ සම්පත් පහළ වෙන්නෙ ...   
188  23804        භික්ෂුවකට අකැප දේ කළ නා හිමි නමක් සිපිරිගෙට   

                                           description  
653  රාජ්‍ය සේවයේ වැටුප් ඉහළ නැංවීම සඳහා එළැඹෙන අයව...  
230  ඉදිරි දින කිහිපයක් සඳහා විදුලිය කප්පාදු කිරීමට...  
70   ආදරවන්තයින්ගේ දිනය හෙටට (14)ට යෙදී තිබෙනවා. ඒ ...  
829  ‘ආණ්ඩුව වෙනස් වුණට හිතූ පැතූ සම්පත් පහළ වෙන්නෙ...  
188  නීතිවිරෝධී මත්ද්‍රව්‍යයක් ලෙස සැලකෙන හෙරොයින් ...  
